# Webscraping of Toronto districts

## Part 1 : data on boroughs

### Importation of libraries

In [42]:
import requests 
from bs4 import BeautifulSoup
import pandas as pd

### Scraping of the relevant Wikipedia page

In [43]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')

### Data formatting

In [44]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [45]:
# Size of the data
df.shape

(103, 3)

## Part 2 - Find longitude and latitude 

In [47]:
#Ajout des colonnes à compléter
df['Latitude'] = 0
df['Longitude'] = 0
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,0,0
1,M4A,North York,Victoria Village,0,0
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",0,0
3,M6A,North York,"Lawrence Manor, Lawrence Heights",0,0
4,M7A,Queen's Park,Ontario Provincial Government,0,0


On fait la requête en utilisant l'API google pour localiser (les exemples donnés dans le cours ne semblent pas convenir). Cela nécessite une clef API qui n'est pas rendue publique car donne le droit à un nombre limité de requêtes.

In [48]:
import requests


for cp in df['PostalCode']:
    req = 'https://maps.googleapis.com/maps/api/geocode/json?address='+cp+', Toronto&key='+API_KEY_GOOGLE
    r = requests.get(req)
    r = r.json()
    df.loc[df['PostalCode'] == cp, 'Latitude'] = r['results'][0]['geometry']['location']['lat']
    df.loc[df['PostalCode'] == cp, 'Longitude'] = r['results'][0]['geometry']['location']['lng']
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
